# Metrics calculation with *AISdb*

Import necessary packages and define a seudo track:

In [1]:
import aisdb
import numpy as np
from aisdb.gis import dt_2_epoch
from datetime import datetime

y1, x1 = 44.57039426840729, -63.52931373766157
y2, x2 = 44.51304767533133, -63.494075674952555
y3, x3 = 44.458038982492134, -63.535634138077945
y4, x4 = 44.393941339104074, -63.53826396955358
y5, x5 = 44.14245580737021, -64.16608964280064

t1 = dt_2_epoch( datetime(2021, 1, 1, 1) )
t2 = dt_2_epoch( datetime(2021, 1, 1, 2) )
t3 = dt_2_epoch( datetime(2021, 1, 1, 3) )
t4 = dt_2_epoch( datetime(2021, 1, 1, 4) )
t5 = dt_2_epoch( datetime(2021, 1, 1, 7) )

# Create a sample track
tracks_short = [
    dict(
        lon=np.array([x1, x2, x3, x4, x5]),
        lat=np.array([y1, y2, y3, y4, y5]),
        time=np.array([t1, t2, t3, t4, t5]),
        mmsi=123456789,
        dynamic=set(['lon', 'lat', 'time']),
        static=set(['mmsi'])
    )
]

## Haversine Distance

*AISdb* includes a function called `aisdb.gis.delta_meters` that calculates the Haversine distance in meters between consecutive positions within a vessel track.

In [2]:
# Calculate the Haversine distance
for track in tracks_short:
    print(aisdb.gis.delta_meters(track))

[ 6961.401286    6948.59446128  7130.40147082 57279.94580704]


## Vessel Speed

In *AISdb*, the speed of a vessel is calculated using the `aisdb.gis.delta_knots` function, which computes the speed over ground (SOG) in knots between consecutive positions within a given track. 

In [3]:
# Calculate the vessel speed in knots
for track in tracks_short:
    print(aisdb.gis.delta_knots(track))

[3.7588560005768947 3.7519408684140214 3.8501088005116215
 10.309565520121597]


## Coast, Shore and Ports

### Distance from shore

In [5]:
from aisdb.webdata.shore_dist import ShoreDist

with ShoreDist(data_dir="./") as sdist:
        # Getting distance from shore for each point in the track
        for track in sdist.get_distance(tracks_short):
            assert 'km_from_shore' in track.keys()
            assert 'km_from_shore' in track['dynamic']
            print(track['km_from_shore'])

raster-shore.7z: 100%|██████████████████████| 39.9M/39.9M [00:00<00:00, 107MB/s]


[ 1  3  2  9 14]


### Distance from coast

In [6]:
from aisdb.webdata.shore_dist import CoastDist

with CoastDist(data_dir="./") as cdist:
        # Getting distance from the coast for each point in the track
        for track in cdist.get_distance(tracks_short):
            assert 'km_from_coast' in track.keys()
            assert 'km_from_coast' in track['dynamic']
            print(track['km_from_coast'])

raster-coast.7z: 100%|██████████████████████| 58.8M/58.8M [00:00<00:00, 112MB/s]


[ 1  3  2  8 13]


### Distance from the nearest port

In [7]:
from aisdb.webdata.shore_dist import PortDist

with PortDist(data_dir="./") as pdist:
        # Getting distance from the port for each point in the track
        for track in pdist.get_distance(tracks_short):
            assert 'km_from_port' in track.keys()
            assert 'km_from_port' in track['dynamic']
            print(track['km_from_port'])

raster-ports.7z: 100%|██████████████████████| 1.26G/1.26G [00:11<00:00, 112MB/s]


[ 4.72144175  7.47747231  4.60478449 11.5642271  28.62511253]
